In [79]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.exceptions import NotFittedError


In [80]:
df = pd.read_json('fruits.json')
df = df['Fruits'].apply(pd.Series) #get the dictionaries in Fruits as columns

#df.insert(3, 'isRound?', [True,True,False,True,True,True,True,True,True,True,True,True,True,False,True])


df


,fruitName,fruitColor,price
0,Apple,Red,1.50
1,Orange,Orange,2.00
2,Banana,Yellow,0.50
3,Grape,Purple,0.10
4,Mango,RedOrange,1.50
5,Lemon,Yellow,1.00
6,Lime,Green,1.00
7,Peach,YellowOrange,1.75
8,Watermelon,"Green, Red",1.00
9,Cherry,Red,0.10


In [86]:
#x = df.drop('price', axis = 1)
x = df['fruitName']
y = df['price']

In [87]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size= 0.3, random_state=1234)

In [88]:
pipelines = {
    'rf': make_pipeline(RandomForestRegressor(random_state=1234)),
    'gb': make_pipeline(GradientBoostingRegressor(random_state=1234)),
    'ridge': make_pipeline(Ridge(random_state=1234)),
    'lasso': make_pipeline (Lasso(random_state=1234)),
    'enet': make_pipeline(ElasticNet(random_state=1234))
}

RandomForestRegressor().get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'criterion': 'squared_error',
 'max_depth': None,
 'max_features': 1.0,
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'monotonic_cst': None,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [89]:
#alpha is learning rate?
hypergrid = {
    'rf': {
        'randomforestregressor__min_samples_split': [2, 4, 6],
        'randomforestregressor__min_samples_leaf': [1, 2, 3]
    },
    'gb': {
        'graientboostingregressor__alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.99]
    },
    'ridge': {
        'ridge__alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.99]
    },
    'lasso': {
        'lasso__alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.99]
    },
    'enet': {
        'elasticnet__alpha': [0.001, 0.005, 0.01, 0.05, 0.1, 0.5, 0.99]
    }
}

In [90]:
fit_models = {}

for algorithm, pipeline in pipelines.items():
    
    #cross validation
    #use all available cpu power
    model = GridSearchCV(pipeline, hypergrid[algorithm], cv = 10, n_jobs = -1) 

    try:
        model.fit(x_train, y_train)
        fit_models[algorithm] = model
    except NotFittedError as e:
        print(repr(e))

    


ValueError: 
All the 90 fits failed.
It is very likely that your model is misconfigured.
You can try to debug the error by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
9 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/jli28/.pyenv/versions/3.11.1/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 866, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/jli28/.pyenv/versions/3.11.1/lib/python3.11/site-packages/sklearn/base.py", line 1389, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/Users/jli28/.pyenv/versions/3.11.1/lib/python3.11/site-packages/sklearn/pipeline.py", line 662, in fit
    self._final_estimator.fit(Xt, y, **last_step_params["fit"])
  File "/Users/jli28/.pyenv/versions/3.11.1/lib/python3.11/site-pac